In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,159 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,320 kB]
Hi

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

def accept_cookies():
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll'))
        )
        cookie_button.click()
    except Exception as e:
        print(f"Could not accept cookies: {e}")

def load_all_articles(limit=1):
    press_count = 0
    while press_count < limit:
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'fnStaticSeachShowMore'))
            )
            load_more_button.click()
            press_count += 1
            print(f"'Hent flere' button clicked {press_count} time(s).")
            time.sleep(2)
        except Exception as e:
            print(f"Error while loading more articles or no more 'Hent flere' button: {e}")
            break


def scrape_article_data(article_url):
    driver.get(article_url)
    time.sleep(3)

    # Extract content
    try:
        header = driver.find_element(By.CLASS_NAME, 'art-title').text
        subheader = driver.find_element(By.CLASS_NAME, 'art-subtitle').text
        date_element = driver.find_element(By.CSS_SELECTOR, "span[data-updatetime='true']")
        date = date_element.get_attribute('data-timestamp') if date_element else ''

        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.article-bodytext p')
        content = ' '.join([p.text for p in paragraphs])
    except Exception as e:
        print(f"Error extracting article: {e}")
        header, subheader, date, content = '', '', '', 'Error extracting content or paywall'

    return {
        'url': article_url,
        'header': header,
        'subheader': subheader,
        'date': date,
        'content': content,
        'search_term': term
    }

# Main scraping function
def scrape_ekstra_bladet(search_term):
    base_url = f"https://ekstrabladet.dk/find/?q={search_term}"
    driver.get(base_url)

    accept_cookies()
    load_all_articles()

    # Extract article links
    article_data = []
    article_elements = driver.find_elements(By.CSS_SELECTOR, "div.flex-item.margin-l--b.width-1of1 a")
    article_links = [article.get_attribute('href') for article in article_elements]

    print(f"Found {len(article_links)} articles for term '{search_term}'")

    # Scrape each article
    for link in article_links:
        article_info = scrape_article_data(link)
        article_data.append(article_info)
        print(f"Scraped: {article_info['header']}")

    return article_data

# Scrape for each term and store results
all_data = []
for term in search_terms:
    articles = scrape_ekstra_bladet(term)
    all_data.extend(articles)

df = pd.DataFrame(all_data)
df.to_csv('ekstra_bladet_articles.csv', index=False)

driver.quit()


'Hent flere' button clicked 1 time(s).
Found 60 articles for term 'ansigtsgenkendelse'
Scraped: Købmænd vil bruge omdiskuteret greb i kampen mod butikstyvene
Scraped: Dan Rachlin har fået nok: - I går topper idiotien
Scraped: FCK tager afstand fra fan-banner
Scraped: Divisions-direktør efter derbyballade: Et samfundsproblem
Scraped: Undersøger ansigtsgenkendelse i hele Superligaen
Scraped: Ingen frihed før ...
Scraped: Anklaget for mishandling og psykisk vold: Hun var jaloux
Scraped: Politiets kontroversielle værktøj er blevet godkendt
Scraped: Politiet indrømmer: Nyt kontroversielt 'våben' på vej
Scraped: Supermarkeder vil fange tyve med ansigtsgenkendelse
Scraped: På ferie med sin drømmemand
Scraped: Kamera-eksplosion: Så meget bliver du overvåget
Scraped: Man kan overvåge alle
Scraped: Politiet kræver nyt våben mod kriminalitet
Scraped: Facebook-ejer betaler 10 mia. i kæmpe forlig
Scraped: Ansigtsgenkendelse kan fremover bruges i pædofilisager
Scraped: Dennis Knudsen: Så meget har d

In [ ]:
from google.colab import files
files.download('ekstra_bladet_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,url,header,subheader,date,content,search_term
0,https://ekstrabladet.dk/nyheder/samfund/koebma...,Købmænd vil bruge omdiskuteret greb i kampen m...,Antallet af butikstyverier har været støt stig...,2024-10-03T16:06:59+02:00,"23 pakker rejer, seks pakker blæksprutte, fire...",ansigtsgenkendelse
1,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Dan Rachlin har fået nok: - I går topper idiotien,FCK-fan Dan Rachlin var med til at buhe ad med...,2024-09-24T19:30:13+02:00,"FCK vandt mandag 2-0 over AaB, men efterfølgen...",ansigtsgenkendelse
2,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,FCK tager afstand fra fan-banner,"FCK håber på, at ansigtsgodkendelse kan være m...",2024-09-24T13:46:34+02:00,Mandag var den gal igen i Parken. Der var mark...,ansigtsgenkendelse
3,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Divisions-direktør efter derbyballade: Et samf...,"Claus Thomsen, der er direktør i Divisionsfore...",2024-09-12T20:13:44+02:00,Vold ved fodboldkampe har de sidste uger igen ...,ansigtsgenkendelse
4,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Undersøger ansigtsgenkendelse i hele Superligaen,"Brøndby har haft ansigtsgenkendelse i fem år, ...",2024-09-12T17:02:00+02:00,Skal der indføres ansigtskendelse på alle 12 s...,ansigtsgenkendelse
...,...,...,...,...,...,...
174,https://ekstrabladet.dk/krimi/overvaagning-paa...,Overvågning på vej i kriminelle områder,300 kameraer skal opsættes i løbet af det komm...,2020-08-19T09:37:33+02:00,Der er nu 50 nye kameraer på vej ud i de dansk...,tryghedsskabende kamera
175,https://ekstrabladet.dk/krimi/her-opsaetter-po...,Her opsætter politiet nye kameraer: - Det er j...,Herning og Silkeborg er blevet udstyret med i ...,2019-01-14T21:27:07+01:00,I Herning og Silkeborg vil man øge trygheden f...,tryghedsskabende kamera
176,https://ekstrabladet.dk/nyheder/samfund/pape-e...,Pape efter post-boykot: Flere kameraer i Volls...,Justitsministeren vil have mere overvågning i ...,2017-06-13T15:25:31+02:00,Justitsminister Søren Pape Poulsen (K) har bed...,tryghedsskabende kamera
177,https://ekstrabladet.dk/nationen/article407116...,"Overvåg mig mere, tak","Ny PET-rapport viser, at danskerne er vilde me...",2011-10-20T11:13:41+02:00,I dag bliver rapporten ’Sikkerhed og tryghed p...,tryghedsskabende kamera


In [ ]:
df['merged_content'] = df['header'] + ' ' + df['subheader'] + ' ' + df['content']

In [ ]:
# Keep only the part of 'merged_content' before 'Kongeparret - Mary og Frede'
df['merged_content'] = df['merged_content'].str.split('Kongeparret - Mary og Frede', n=1).str[0]


<ipython-input-11-0a6c4b8323c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['merged_content'] = filtered_df['merged_content'].str.split('Kongeparret - Mary og Frede', n=1).str[0]


In [ ]:

df['date'] = df['date'].str[:10]


<ipython-input-12-ec052f319464>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['date'] = filtered_df['date'].str[:10]


In [ ]:
df

,url,header,subheader,date,content,search_term,merged_content
0,https://ekstrabladet.dk/nyheder/samfund/koebma...,Købmænd vil bruge omdiskuteret greb i kampen m...,Antallet af butikstyverier har været støt stig...,2024-10-03,"23 pakker rejer, seks pakker blæksprutte, fire...",ansigtsgenkendelse,Købmænd vil bruge omdiskuteret greb i kampen m...
1,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Dan Rachlin har fået nok: - I går topper idiotien,FCK-fan Dan Rachlin var med til at buhe ad med...,2024-09-24,"FCK vandt mandag 2-0 over AaB, men efterfølgen...",ansigtsgenkendelse,Dan Rachlin har fået nok: - I går topper idiot...
2,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,FCK tager afstand fra fan-banner,"FCK håber på, at ansigtsgodkendelse kan være m...",2024-09-24,Mandag var den gal igen i Parken. Der var mark...,ansigtsgenkendelse,"FCK tager afstand fra fan-banner FCK håber på,..."
3,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Divisions-direktør efter derbyballade: Et samf...,"Claus Thomsen, der er direktør i Divisionsfore...",2024-09-12,Vold ved fodboldkampe har de sidste uger igen ...,ansigtsgenkendelse,Divisions-direktør efter derbyballade: Et samf...
4,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,Undersøger ansigtsgenkendelse i hele Superligaen,"Brøndby har haft ansigtsgenkendelse i fem år, ...",2024-09-12,Skal der indføres ansigtskendelse på alle 12 s...,ansigtsgenkendelse,Undersøger ansigtsgenkendelse i hele Superliga...
...,...,...,...,...,...,...,...
174,https://ekstrabladet.dk/krimi/overvaagning-paa...,Overvågning på vej i kriminelle områder,300 kameraer skal opsættes i løbet af det komm...,2020-08-19,Der er nu 50 nye kameraer på vej ud i de dansk...,tryghedsskabende kamera,Overvågning på vej i kriminelle områder 300 ka...
175,https://ekstrabladet.dk/krimi/her-opsaetter-po...,Her opsætter politiet nye kameraer: - Det er j...,Herning og Silkeborg er blevet udstyret med i ...,2019-01-14,I Herning og Silkeborg vil man øge trygheden f...,tryghedsskabende kamera,Her opsætter politiet nye kameraer: - Det er j...
176,https://ekstrabladet.dk/nyheder/samfund/pape-e...,Pape efter post-boykot: Flere kameraer i Volls...,Justitsministeren vil have mere overvågning i ...,2017-06-13,Justitsminister Søren Pape Poulsen (K) har bed...,tryghedsskabende kamera,Pape efter post-boykot: Flere kameraer i Volls...
177,https://ekstrabladet.dk/nationen/article407116...,"Overvåg mig mere, tak","Ny PET-rapport viser, at danskerne er vilde me...",2011-10-20,I dag bliver rapporten ’Sikkerhed og tryghed p...,tryghedsskabende kamera,"Overvåg mig mere, tak Ny PET-rapport viser, at..."


In [ ]:
from google.colab import files
import pandas as pd

filtered_df.to_csv('ekstra_bladet_articles_clean.csv', index=False)
files.download('ekstra_bladet_articles_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>